In [1]:
#https://github.com/openvinotoolkit/openvino/tree/master/docs/template_extension/new

In [ ]:
#安装sudo,编译文件中需要
!apt-get update
!apt-get install sudo

In [ ]:
#下载openvino
!git clone https://github.com/openvinotoolkit/openvino.git
#git clone -b 2023.0.1 https://gitee.com/openvinotoolkit-prc/openvino.git
%cd openvino
!git submodule update --init --recursive#这一步必须做，处理第三方库的

In [ ]:
#更新cmake
!pip install cmake --upgrade

In [ ]:
#安装环境
!y|./install_build_dependencies.sh

In [ ]:
#配置PYTHON
#进入到python路径下，安装python的环境
%cd /kaggle/working/openvino/src/bindings/python/src/compatibility/openvino/
!pip install -r requirements-dev.txt

In [ ]:
#准备编译
%cd /kaggle/working/openvino/
!mkdir build 
%cd build

In [ ]:
!cmake -DCMAKE_BUILD_TYPE=Release -DENABLE_PYTHON=ON  -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.8m.so  -DPYTHON_INCLUDE_DIR=/usr/include/python3.8-DENABLE_INTEL_GPU=OFF -DENABLE_OV_ONNX_FRONTEND=ON -DENABLE_OV_TENSORFLOW_FRONTEND=OFF ..

In [ ]:
#编译命令
!cmake -DCMAKE_BUILD_TYPE=Release  \
        -DENABLE_PYTHON=ON  \  #是否编译python API，使用PYTHON编译选项后才会生成python文件夹,

        #指定python版本，可以全部删除
        -DPYTHON_EXECUTABLE=`which python3.6`  \ 
        -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.6m.so  \
        -DPYTHON_INCLUDE_DIR=/usr/include/python3.6  \
        

        -DENABLE_INTEL_GPU=OFF  \# 是否编译GPU的计算
         #是否编译onnx的版本
        -DENABLE_OV_ONNX_FRONTEND=ON 
        -DENABLE_OV_TENSORFLOW_FRONTEND=OFF ..
!make --jobs=$(nproc --all)

#。。。。。。。。。。。。。。。编译过程。。。。。。。。。。。。。。。。。。。。。。。。。。。

In [ ]:
#编译完成以后，模块就放在了build文件夹里面
#做自定义编译,编译完成后，一些需要的文件才会出现再build文件夹中,都是cmake文件
#非常耗时的编译过程

In [ ]:
!mkdir install_openvino
#安装   给定已经build路径，和一个空文件路径用于存放安装文件
!cmake --install <BUILDDIR> --prefix <INSTALLDIR>

In [ ]:
#设置OpenVINO环境变量，要看是否真正执行了，会显示成功
%cd install_openvino
!./setupvars.sh

/kaggle/working/install_openvino

[setupvars.sh] OpenVINO environment initialized

In [ ]:
#做完上面工作就可进行编译，可以使用下面的编译文件了，就是模板文件

In [ ]:
%cd /kaggle/working/openvino/src/core/template_extension/new
!mkdir build
%cd build
#编译时需要用到openvino的头文件，就要把他加入到工程中，注意这里的OpenVINO_DIR路径是安装的路径
!cmake -DOpenVINO_DIR=/kaggle/working/install_openvino/runtime/cmake ../
!cmake --build .

    -- Configuring done
    -- Generating done
    -- Build files have been written to: /kaggle/working/openvino/docs/template_extension/new/build
    Consolidate compiler generated dependencies of target openvino_template_extension
    [100%] Built target openvino_template_extension
  
  
 编译生成libopenvino_template_extension.so

In [ ]:
#这个用的是pip安装的就行了，有命名冲突的问题
from openvino.runtime import Core
#可能会有动态库文件无法加载的问题，要把../install_openvino/runtime/lib/intel64文件放到动态库中
core = Core()
core.add_extension('libopenvino_template_extension.so')
#如果出现无法加载的问题，可能还需要他的绝对路径，或者需要把他放到动态库中